In [ ]:
import os
import sys
import os
import random
import json
import argparse

files = ['data/conllpp_dev.txt', 'data/conllpp_test.txt', 'data/conllpp_train.txt']
#for file in files:
#  f = open(file, 'r', encoding='utf-8')
#  for line in f.readlines() + ['']:
#    print(line)


In [ ]:
def get_sentences(filename):
  sentences = []
  tokens = []
  labels = []
  file = open(filename, 'r', encoding='utf-8')
  for line in file.readlines() + ['']:
    if len(line) == 0 or line.startswith('-DOCSTART-') or line.isspace():
      if len(tokens) > 0:
        sentences.append((tokens, labels))
      tokens = []
      labels = []
    else:
      splits = line.strip().split()
      token, label = splits[0], splits[-1]
      tokens.append(token)
      labels.append(label)
      #print(f"Token: {token}")
      #print(f"Label: {label}")
  return sentences

In [ ]:
sentences = get_sentences(files[2])


In [ ]:
#sentences[0][1][0]
len(sentences)
full_token_list = []
full_label_list = []
for sentence in sentences:
    token_list = sentence[0]
    label_list = sentence[1]
    full_token_list.extend(token_list)
    full_label_list.extend(label_list)
print(f"Token list length: {len(full_token_list)} and label list length: {len(full_label_list)}")



In [ ]:
import pandas as pd
df = pd.DataFrame(list(zip(full_token_list, full_label_list)),
               columns =['Tokens', 'Labels'])
df

In [ ]:
unique_label_list = df.Labels.unique()
unique_label_list
print(unique_label_list)
print(type(unique_label_list))

In [ ]:
unique_token_list = df.Tokens.unique()
print(len(unique_token_list))
print(type(unique_token_list))


In [ ]:
labels_counts = df['Labels'].value_counts()
labels_counts


In [ ]:
#unique_label_list is --> ['B-ORG', 'O', 'B-MISC', 'B-PER', 'I-PER', 'B-LOC', 'I-ORG',
#       'I-MISC', 'I-LOC']
print(f"Label: {unique_label_list[1]} and its count: {labels_counts[unique_label_list[1]]}")



In [ ]:
# A matrix will contain the label transition probabilities. P(T_i, T_i-1)
# P(O|B-ORG) = C(B-ORG, O) / C(B-ORG)
# Total labels: 203621
# Need: P(c_o, c_b_loc), P(c_o, c_b_per), P(c_o, c_b_org) ... for all.
# O->B_LOC, O->B_PER etc.
import numpy as np

# first create 9x9 matrix without <s>
a_w_out_start = np.zeros((len(unique_label_list), len(unique_label_list))) # A matrix
#print(a)
for t_index, t in enumerate(unique_label_list):
    for t_minus_1_index, t_minus_1 in enumerate(unique_label_list):
        count_t_minus_1_t = 0
        for sentence in sentences:
            label_list = sentence[1]
            for label_index in range(1, len(label_list)):
                if label_list[label_index] == t:
                    if label_list[label_index - 1] == t_minus_1:
                        count_t_minus_1_t += 1
        # finished all sentences
        probability = count_t_minus_1_t / labels_counts[unique_label_list[t_minus_1_index]]
        #print(f"t_index: {t_index}, t_i-1 index: {t_minus_1_index}")
        a_w_out_start[t_index][t_minus_1_index] = probability

print(f"a_w_out_start: {a_w_out_start}")
print("-----------------------------------------------------")

# need start matrix that includes <s> as well.
start = np.zeros((1, (len(unique_label_list)))) 
for t_index, t in enumerate(unique_label_list):
    count_t_start = 0
    for sentence in sentences:
        label_list = sentence[1]
        if label_list[0] == t:
            count_t_start += 1
    probability = count_t_start / labels_counts[unique_label_list[t_index]]
    start[0][t_index] = probability
print(f"Only start matrix: {start}")
print("-----------------------------------------------------")
# concatanate
a = np.concatenate((start, a_w_out_start))
print(f"A matrix: {a}")

In [ ]:
np.set_printoptions(suppress=True)
print(a)

In [ ]:
# B matrix: emmision probabilities
# P(w_i|t_i) = C(t_i, w_i) / C(t_i)
# P(German|B-MISC) = C(B-MISC, German) / C(B-MISC)
# NOTE: Takes a lot of time

b = np.zeros((len(unique_token_list), len(unique_label_list))) # B matrix
for token_index, token in enumerate(unique_token_list):
    for label_index, label in enumerate(unique_label_list):
        print(f"Working on {token_index}. token: {token} and {label_index}. label: {label}")
        count_t_i_w_i = 0
        for index, token_input in enumerate(full_token_list):
            if token_input == token and label == full_label_list[index]:
                count_t_i_w_i += 1
        probability = count_t_i_w_i / labels_counts[unique_label_list[label_index]]
        b[token_index][label_index] = probability
print(b)

In [ ]:
np.savetxt('b_matrix.txt', b)
b_loaded = np.loadtxt('b_matrix.txt')
b_loaded

In [ ]:
np.set_printoptions(suppress=True)
np.savetxt('b_matrix_scientific.txt', b)
b_loaded_sci = np.loadtxt('b_matrix_scientific.txt')
b_loaded_sci

In [ ]:
B = np.transpose(b)
B.shape

In [ ]:
B

In [ ]:
# convert token np array to list
unique_token_list_lower = []
for token in unique_token_list:
    unique_token_list_lower.append(token.lower())
try:
    token_index = unique_token_list_lower.index("japan")
    print(ind)
except Exception as exc:
    print(f"exception: {exc}")

In [ ]:
# Viterbi Algorithm
# given input sentence: Janet will back the bill
# word_array, T = ['Janet', 'will', 'back', 'the', 'bill']
# unique_label_list == N
def viterbi(N, T):  
    viterbi_matrix = np.zeros((len(N), len(T))) # B matrix
    backpointer = np.zeros((len(N), len(T)))
    for s_index, s in enumerate(N):
        try:
            item_index = unique_token_list_lower.index(T[0].lower())
        except Exception as exc:
            #raise Exception(f"Item {T[0].lower()} is not found in the token list. Exception: {exc}")
            item_index = -1
        # if the word is found in B matrix 
        if item_index > -1:
            viterbi_matrix[s_index][0] = a[0][s_index] * B[s_index][item_index]
        else:
            viterbi_matrix[s_index][0] = 0
        backpointer[s_index][0] = 0
    # initialization ends. 
    # recursion step
    for t_index, t in enumerate(T[1:]):
        for s_index, s in enumerate(N):
            # maximum probability viterbi step
            max_prob = 0.0
            for s_prime_index, s_prime in enumerate(N):
                try:
                    item_index = unique_token_list_lower.index(T[t_index + 1].lower())
                except Exception as exc:
                    #raise Exception(f"Item {T[t_index].lower()} is not found in the token list. Exception: {exc}")
                    item_index = -1
                # if the word is found in B matrix
                if item_index > -1:
                    b_s_o_t = B[s_index][item_index]
                else:
                    b_s_o_t = 0
                calculated_probability = viterbi_matrix[s_prime_index][t_index] * a_w_out_start[s_prime_index][s_index] * b_s_o_t # it is not t_index - 1 since enumerate make index start with 0
                if calculated_probability > max_prob:
                    max_prob = calculated_probability
            viterbi_matrix[s_index][t_index + 1] = max_prob # # it is t_index + 1 since enumerate make index start with 0
            backpointer[s_index][t_index + 1] = max_prob # it is t_index + 1 since enumerate make index start with 0
            # maximum probability viterbi step ends
    # find bestpathprob & best_state_dict
    best_state_list = []
    best_prob_list = []
    for t_index, t in enumerate(T):
        best_state = 0.0
        best_state_list.append("Not_Assigned")
        best_prob_list.append(best_state)
        for s_index, s in enumerate(N):
            if viterbi_matrix[s_index][t_index] > best_state:
                best_state = viterbi_matrix[s_index][t_index]
                best_prob_list[t_index] = viterbi_matrix[s_index][t_index]
                best_state_list[t_index] = s
        if best_state == 0.0:
            best_state_list[t_index] = "O" # Assumption: Token is outside of any span of interest, thus mark it as O
    return best_prob_list, best_state_list



In [ ]:
#SOCCER NN I-NP O
#- : O O
#JAPAN NNP I-NP B-LOC
#GET VB I-VP O
#LUCKY NNP I-NP O
#WIN NNP I-NP O
#, , O O
#CHINA NNP I-NP B-LOC
#IN IN I-PP O
#SURPRISE DT I-NP O
#DEFEAT NN I-NP O
#. . O O
np.set_printoptions(suppress=False)
T = ['SOCCER', '-', 'JAPAN', 'GET', 'LUCKY', 'WIN', ',', 'CHINA', 'IN', 'SURPRISE', 'DEFEAT', '.']
best_prob_dict, best_state_dict = viterbi(unique_label_list, T)

print(f"best_prob_dict is: {best_prob_dict}")
print(f"best_state_dict is: {best_state_dict}")

In [ ]:
# Get test file
test_sentences = get_sentences(files[1])

In [ ]:

len(test_sentences)
test_token_list = []
test_label_list = []
for sentence in test_sentences:
    token_list = sentence[0]
    label_list = sentence[1]
    test_token_list.extend(token_list)
    test_label_list.extend(label_list)
print(f"Test token list length: {len(test_token_list)} and test label list length: {len(test_label_list)}")


In [ ]:

df_test = pd.DataFrame(list(zip(test_token_list, test_label_list)),
               columns =['Tokens', 'Labels'])
df_test

In [ ]:
labels_counts_test = df_test['Labels'].value_counts()
labels_counts_test

In [ ]:
# convert label np array to list
unique_label_python_list = []
for label in unique_label_list:
    unique_label_python_list.append(label)
try:
    label_index = unique_label_python_list.index("B-LOC")
    print(label_index)
except Exception as exc:
    print(f"exception: {exc}")

In [ ]:
confusion_matrix

In [ ]:
confusion_matrix = np.zeros((len(unique_label_list), len(unique_label_list)))
for sentence in test_sentences:
    token_list = sentence[0]
    label_list = sentence[1]
    #print(token_list)
    best_prob_list, prediction_list = viterbi(unique_label_list, token_list)
    #print(f"best_prob_list: {best_prob_list}")
    #print(f"Prediction values: {prediction_list}, len: {len(prediction_list)}")
    #print(f"Y-test values: {label_list}, len: {len(label_list)}")
    for index, y_item in enumerate(label_list):
        y_index = unique_label_python_list.index(y_item) # locate y_test value in confusion matrix
        prediction_index = unique_label_python_list.index(prediction_list[index]) # locate y_pred value in confusion matrix
        confusion_matrix[prediction_index][y_index] += 1
        #print(f"prediction_index is {prediction_index}, y_index is {y_index}")
    

In [ ]:
# create matrix for each label with corresponding precision recall f1-score
eval_matrix = np.zeros((len(unique_label_list), 3))
tp_total = 0
tn_total = 0
fp_total = 0
fn_total = 0
precision_total = 0
recall_total = 0
f1_total = 0
for index, label in enumerate(unique_label_list):
    tp = confusion_matrix[index][index]
    tn_matrix = numpy.delete(confusion_matrix, (index), axis=0)
    tn_matrix = numpy.delete(tn_matrix, (index), axis=1)
    tn = np.sum(tn_matrix)
    fp = np.sum(confusion_matrix[index]) - tp
    fn = np.sum(confusion_matrix[ : , index]) - tp
    tp_total += tp
    tn_total += tn
    fp_total += fp
    fn_total += fn
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * tp / (2 * tp + fp + fn)
    precision_total += precision
    recall_total += recall
    f1_total += f1
    eval_matrix[index][0] = precision
    eval_matrix[index][1] = recall
    eval_matrix[index][2] = f1
    
print(f"Aşağıdaki matris için verilen etiketler (yukarıdan aşağıya): \n{np.transpose(unique_label_list)}")  
print(f"  kesinlik   duyarlılık       f1 \n{eval_matrix}")
micro_precision = tp_total / (tp_total + fp_total)
micro_recall = tp_total / (tp_total + fn_total)
micro_f1 = 2 * tp_total / (2 * tp_total + fp_total + fn_total) 
print(f"           kesinlik                duyarlılık                    f1")
print(f"Mikro ort: {micro_precision}       {micro_recall}        {micro_f1}")   
print(f"Makro ort: {precision_total/len(unique_label_list)}       {recall_total/len(unique_label_list)}       {f1_total/len(unique_label_list)}")   

weighted_precision_total = 0
weighted_recall_total = 0
weighted_f1_total = 0
for index, label in enumerate(unique_label_list):
    weighted_precision_total += eval_matrix[index][0] * labels_counts_test[label]
    weighted_recall_total += eval_matrix[index][1] * labels_counts_test[label]
    weighted_f1_total += eval_matrix[index][2] * labels_counts_test[label]
print(f"Ağırlıklı F1: {weighted_precision_total/np.sum(labels_counts_test)}       {weighted_recall_total/np.sum(labels_counts_test)}       {weighted_f1_total/np.sum(labels_counts_test)}")   
    

# 2. Dereceden HMM 

In [ ]:
# HMM 2. degree
# A matrix will contain the label transition probabilities. P(T_i | T_i-1, T_i-2)

# 3d matrix:
a_second_w_out_start = np.zeros((len(unique_label_list), len(unique_label_list), len(unique_label_list))) # 3d A matrix

for t_index, t in enumerate(unique_label_list):
    for t_minus_1_index, t_minus_1 in enumerate(unique_label_list):
        for t_minus_2_index, t_minus_2 in enumerate(unique_label_list):
            # first count C(T_i-1, T_i-2)
            count_t_i_minus_1_and_t_i_minus_2 = 0
            for sentence in sentences:
                label_list = sentence[1]
                for label_index in range(1, len(label_list)):
                    if label_list[label_index - 1] == t_minus_1:
                        if label_list[label_index - 2] == t_minus_2:
                            count_t_i_minus_1_and_t_i_minus_2 += 1
            
            # then count C(T_i-1, T_i-2, T_i)
            count_trigram = 0
            for sentence in sentences:
                label_list = sentence[1]
                for label_index in range(1, len(label_list)):
                    if label_list[label_index] == t:
                        if label_list[label_index - 1] == t_minus_1:
                            if label_list[label_index - 2] == t_minus_2:
                                count_trigram += 1
            # finished all sentences
            if count_t_i_minus_1_and_t_i_minus_2 == 0:
                #print('Division by zero!')
                probability = 0.0
            else:
                probability = count_trigram / count_t_i_minus_1_and_t_i_minus_2 
            #print(f"t_index: {t_index}, t_i-1 index: {t_minus_1_index}")
            a_second_w_out_start[t_index, t_minus_1_index, t_minus_2_index] = probability
                

print(f"a_second_w_out_start: {a_second_w_out_start}")
print("-----------------------------------------------------")


In [ ]:
# Viterbi Algorithm
# given input sentence: Janet will back the bill
# word_array, T = ['Janet', 'will', 'back', 'the', 'bill']
# unique_label_list == N
def viterbi_2d(N, T):  
    viterbi_matrix = np.zeros((len(N), len(T))) # B matrix
    backpointer = np.zeros((len(N), len(T)))
    for s_index, s in enumerate(N):
        try:
            item_index = unique_token_list_lower.index(T[0].lower())
        except Exception as exc:
            #raise Exception(f"Item {T[0].lower()} is not found in the token list. Exception: {exc}")
            item_index = -1
        # if the word is found in B matrix 
        if item_index > -1:
            viterbi_matrix[s_index][0] = a[0][s_index] * B[s_index][item_index]
        else:
            viterbi_matrix[s_index][0] = 0
        backpointer[s_index][0] = 0
        
    # initialization ends. 
    # recursion step
    for t_index, t in enumerate(T[1:]):
        # second col of viterbi matrix
        if t_index == 0:
            for s_index, s in enumerate(N):
                # maximum probability viterbi step
                max_prob = 0.0
                for s_prime_index, s_prime in enumerate(N):
                    try:
                        item_index = unique_token_list_lower.index(T[t_index + 1].lower())
                    except Exception as exc:
                        #raise Exception(f"Item {T[t_index].lower()} is not found in the token list. Exception: {exc}")
                        item_index = -1
                    # if the word is found in B matrix
                    if item_index > -1:
                        b_s_o_t = B[s_index][item_index]
                    else:
                        b_s_o_t = 0
                    calculated_probability = viterbi_matrix[s_prime_index][t_index] * a_w_out_start[s_prime_index][s_index] * b_s_o_t # it is not t_index - 1 since enumerate make index start with 0
                    if calculated_probability > max_prob:
                        max_prob = calculated_probability
                viterbi_matrix[s_index][t_index + 1] = max_prob # # it is t_index + 1 since enumerate make index start with 0
                backpointer[s_index][t_index + 1] = max_prob # it is t_index + 1 since enumerate make index start with 0
                # maximum probability viterbi step ends
        # after second col
        else:
            # find  the best path at t_index - 1
            
            best_state = 0.0
            best_label = ""
            for s_index, s in enumerate(N):
                if viterbi_matrix[s_index][t_index - 1] > best_state:
                    best_state = viterbi_matrix[s_index][t_index - 1]
                    best_label = s
            if best_state == 0.0:
                best_label = "O" 
                
            best_label_index = unique_label_python_list.index(best_label) # best label index at t-2   
            for s_index, s in enumerate(N):
                # maximum probability viterbi step
                max_prob = 0.0
                for s_prime_index, s_prime in enumerate(N):
                    try:
                        item_index = unique_token_list_lower.index(T[t_index + 1].lower())
                    except Exception as exc:
                        #raise Exception(f"Item {T[t_index].lower()} is not found in the token list. Exception: {exc}")
                        item_index = -1
                    # if the word is found in B matrix
                    if item_index > -1:
                        b_s_o_t = B[s_index][item_index]
                    else:
                        b_s_o_t = 0
                    calculated_probability = viterbi_matrix[s_prime_index][t_index] * a_second_w_out_start[s_prime_index][s_index][best_label_index] * b_s_o_t # it is not t_index - 1 since enumerate make index start with 0
                    if calculated_probability > max_prob:
                        max_prob = calculated_probability
                viterbi_matrix[s_index][t_index + 1] = max_prob # # it is t_index + 1 since enumerate make index start with 0
                backpointer[s_index][t_index + 1] = max_prob # it is t_index + 1 since enumerate make index start with 0
                # maximum probability viterbi step ends
    # find bestpathprob & best_state_dict
    best_state_list = []
    best_prob_list = []
    for t_index, t in enumerate(T):
        best_state = 0.0
        best_state_list.append("Not_Assigned")
        best_prob_list.append(best_state)
        for s_index, s in enumerate(N):
            if viterbi_matrix[s_index][t_index] > best_state:
                best_state = viterbi_matrix[s_index][t_index]
                best_prob_list[t_index] = viterbi_matrix[s_index][t_index]
                best_state_list[t_index] = s
        if best_state == 0.0:
            best_state_list[t_index] = "O" # Assumption: Token is outside of any span of interest, thus mark it as O
    return best_prob_list, best_state_list


In [ ]:
#SOCCER NN I-NP O
#- : O O
#JAPAN NNP I-NP B-LOC
#GET VB I-VP O
#LUCKY NNP I-NP O
#WIN NNP I-NP O
#, , O O
#CHINA NNP I-NP B-LOC
#IN IN I-PP O
#SURPRISE DT I-NP O
#DEFEAT NN I-NP O
#. . O O
np.set_printoptions(suppress=False)
T = ['SOCCER', '-', 'JAPAN', 'GET', 'LUCKY', 'WIN', ',', 'CHINA', 'IN', 'SURPRISE', 'DEFEAT', '.']
best_prob_dict, best_state_dict = viterbi_2d(unique_label_list, T)

print(f"best_prob_dict is: {best_prob_dict}")
print(f"best_state_dict is: {best_state_dict}")

In [ ]:
confusion_matrix = np.zeros((len(unique_label_list), len(unique_label_list)))
for sentence in test_sentences:
    token_list = sentence[0]
    label_list = sentence[1]
    #print(token_list)
    best_prob_list, prediction_list = viterbi_2d(unique_label_list, token_list)
    #print(f"best_prob_list: {best_prob_list}")
    #print(f"Prediction values: {prediction_list}, len: {len(prediction_list)}")
    #print(f"Y-test values: {label_list}, len: {len(label_list)}")
    for index, y_item in enumerate(label_list):
        y_index = unique_label_python_list.index(y_item) # locate y_test value in confusion matrix
        prediction_index = unique_label_python_list.index(prediction_list[index]) # locate y_pred value in confusion matrix
        confusion_matrix[prediction_index][y_index] += 1
        #print(f"prediction_index is {prediction_index}, y_index is {y_index}")
    

In [ ]:
# create matrix for each label with corresponding precision recall f1-score
eval_matrix = np.zeros((len(unique_label_list), 3))
tp_total = 0
tn_total = 0
fp_total = 0
fn_total = 0
precision_total = 0
recall_total = 0
f1_total = 0
for index, label in enumerate(unique_label_list):
    tp = confusion_matrix[index][index]
    tn_matrix = numpy.delete(confusion_matrix, (index), axis=0)
    tn_matrix = numpy.delete(tn_matrix, (index), axis=1)
    tn = np.sum(tn_matrix)
    fp = np.sum(confusion_matrix[index]) - tp
    fn = np.sum(confusion_matrix[ : , index]) - tp
    tp_total += tp
    tn_total += tn
    fp_total += fp
    fn_total += fn
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * tp / (2 * tp + fp + fn)
    precision_total += precision
    recall_total += recall
    f1_total += f1
    eval_matrix[index][0] = precision
    eval_matrix[index][1] = recall
    eval_matrix[index][2] = f1
    
print(f"Aşağıdaki matris için verilen etiketler (yukarıdan aşağıya): \n{np.transpose(unique_label_list)}")  
print(f"  kesinlik   duyarlılık       f1 \n{eval_matrix}")
micro_precision = tp_total / (tp_total + fp_total)
micro_recall = tp_total / (tp_total + fn_total)
micro_f1 = 2 * tp_total / (2 * tp_total + fp_total + fn_total) 
print(f"           kesinlik                duyarlılık                    f1")
print(f"Mikro ort: {micro_precision}       {micro_recall}        {micro_f1}")   
print(f"Makro ort: {precision_total/len(unique_label_list)}       {recall_total/len(unique_label_list)}       {f1_total/len(unique_label_list)}")   

weighted_precision_total = 0
weighted_recall_total = 0
weighted_f1_total = 0
for index, label in enumerate(unique_label_list):
    weighted_precision_total += eval_matrix[index][0] * labels_counts_test[label]
    weighted_recall_total += eval_matrix[index][1] * labels_counts_test[label]
    weighted_f1_total += eval_matrix[index][2] * labels_counts_test[label]
print(f"Ağırlıklı F1: {weighted_precision_total/np.sum(labels_counts_test)}       {weighted_recall_total/np.sum(labels_counts_test)}       {weighted_f1_total/np.sum(labels_counts_test)}")   
    